# Self-supervised learning with fastai

Inspired by [Ayush Thakur's work](https://www.kaggle.com/ayuraj/v2-self-supervised-pretraining-with-swav?scriptVersionId=59516445), I started exploring more about SwAV. The results in the [paper](https://arxiv.org/abs/2006.09882) were quite impressive. Hence, I decided to implement it.

Lucky for me, I found a pytorch implementation. Also, it used fastai. This was like a dream come true. Fastai is my comfort zone. You can find more about the [implementation here](https://keremturgutlu.github.io/self_supervised/). I will highly recommend checking out the documentation. Not just SwAV, the repository has fastai implementations of other state-of-the-art algorithms as well.

I followed [this tutorial](https://keremturgutlu.github.io/self_supervised/04%20-%20training_swav_iwang.html) for creating this notebook that you are reading now. So, if something looks off or doesn't make sense, then please refer the original tutorial.

Lets get started . . .

### Installation & imports

In [2]:
!pip install self-supervised -Uq

In [16]:
from fastai.vision.all import *

from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.vision.swav import *
from self_supervised.vision.byol import *

from sklearn.model_selection import StratifiedKFold
import torchvision.models as models
from cuml.neighbors import NearestNeighbors

### Reading Data

In [4]:
df = pd.read_csv('../input/shopee-product-matching/train.csv')
df.head()

To train the model, we will also need a validation set. I will use simple `StratifiedKFold` technique to split my data into train & validation set.

In [5]:
sk_fold = StratifiedKFold(5)
df['is_valid'] = False
for i, (trn_idx, val_idx) in enumerate(sk_fold.split(df, df.label_group)):
    df.loc[val_idx, 'is_valid'] = True
    break
    
df.groupby('is_valid').label_group.value_counts()

As per the warning, there are some label groups with less than 5 posting. lets see how many such label group we have . . . 

In [6]:
sum(df.label_group.value_counts() < 5)

There are 9620 `label_group` with less than 5 postings. Quite a lot, huh! (lets handle it in version-2).

### Dataloaders

For now, lets create some helper functions to create dataloaders.

In [7]:
def get_x(x): return '../input/shopee-product-matching/train_images/' + x['image']

def get_dls(size, bs, workers=None):
    path = Path('../input/shopee-product-matching/train_images/')
    
    db = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
              get_x = get_x, get_y=ColReader('label_group'),
              splitter=ColSplitter(),
              item_tfms=RandomResizedCrop(size, min_scale=1.))
    dls = db.dataloaders(df, bs=bs, num_workers=workers)
    return dls

Lets create our dataloaders . . . 

In [8]:
bs, resize, size = 24, 256, 224
dls = get_dls(resize, bs)

### Model & Callbacks

Finally, lets initialize our model & SwAV callbacks

In [37]:
from self_supervised.vision.simclr import *
## Model
arch = "resnet50"
encoder = create_encoder(arch, pretrained=True, n_in=3)
model = create_simclr_model(encoder, hidden_size=2048, projection_size=128)

## SwAV callback
K = bs*2**4
aug_pipelines = get_byol_aug_pipelines(size=size)
cbs=[SimCLR(aug_pipelines=aug_pipelines)]

Great, we have our data, model, & also the callbacks. Fastai has this amazing class called `Learner` which put everything together for training.

In [38]:
learn = Learner(dls, model, cbs=cbs)

Before we actually training the model, lets look at some of the samples. 

In [39]:
b = dls.one_batch()
learn._split(b)
# learn('before_batch')
# learn.byol.show(n=5);

### Training 

Time to train the model . . . 

In [40]:
lr, wd = 1e-2, 1e-2
epochs = 5 # try using 40 or 50
learn.unfreeze()
learn.fit_flat_cos(epochs, lr, wd=wd, pct_start=0.5)

In [41]:
save_name = f'simclr_iwang_sz{size}_epc{epochs}'
learn.save(save_name)
torch.save(learn.model.encoder.state_dict(), learn.path/learn.model_dir/f'{save_name}_encoder.pth')
learn.recorder.plot_loss()

In [34]:
ls

In [ ]:
while True:
    pass

This notebook is only intended for learning. To get better score, try using different model architecture, bigger image size, etc.

I can find the inference notebook, [here](https://www.kaggle.com/ankursingh12/shopee-swav-inference)

Hope you enjoyed reading this notebook. If yes, then please consider **upvoting**!